### Initialize

In [39]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [40]:
# Configure path to data folder
DATA_PATH = 'C:/Users/USER/Documents/Portfolio stuff/Fintech_CLV_Churn_Analysis/data/'
SAMPLE_FRACTION = 0.10 # 10% sample for the largest file (order_products__prior)

In [41]:
# Load datasets
orders = pd.read_csv(os.path.join(DATA_PATH, 'orders.csv'))
products = pd.read_csv(os.path.join(DATA_PATH, 'products.csv'))
departments = pd.read_csv(os.path.join(DATA_PATH, 'departments.csv'))

num_lines = sum(1 for line in open(os.path.join(DATA_PATH, 'order_products__prior.csv'))) - 1 # Calculate the number of lines (required for sampling logic)
skip_idx = np.random.choice(np.arange(1, num_lines + 1), size=int(num_lines * (1 - SAMPLE_FRACTION)), replace=False) # Create an index of rows to skip (we skip 90 out of every 100 rows, plus the header)

# Load the sampled data
order_products_prior = pd.read_csv(os.path.join(DATA_PATH, 'order_products__prior.csv'), skiprows=skip_idx)

### Merging the files to create a transaction log (since this is framing the analysis into a fintech frame)

In [54]:
# Merge order line items with product details to get department info.
# Join 'order_products_prior' (the transaction details) with 'products' to associate each line item with its department (the proxy for 'Merchant Vertical').
merged_df = order_products_prior.merge(products[['product_id', 'department_id']], on='product_id', how='left')

In [55]:
# Merge the combined product details with the order metadata (customer ID, time) to create the actual transaction log.
# join with 'orders' to link each transaction (product line item) back to the specific 'user_id' (Customer ID) and the time.
transaction_log = merged_df.merge(orders[['order_id', 'user_id', 'order_number', 'days_since_prior_order', 'order_dow', 'order_hour_of_day', 'eval_set']], on='order_id', how='left')

### Little EDA

In [56]:
print(f"\nInitial Transaction Log Merged. \nShape: {transaction_log.shape}")


Initial Transaction Log Merged. 
Shape: (3243449, 11)


In [57]:
print(transaction_log.info())
transaction_log.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3243449 entries, 0 to 3243448
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   product_id              int64  
 2   add_to_cart_order       int64  
 3   reordered               int64  
 4   department_id           int64  
 5   user_id                 int64  
 6   order_number            int64  
 7   days_since_prior_order  float64
 8   order_dow               int64  
 9   order_hour_of_day       int64  
 10  eval_set                object 
dtypes: float64(1), int64(9), object(1)
memory usage: 272.2+ MB
None


,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,order_number,days_since_prior_order,order_dow,order_hour_of_day,eval_set
0,2,9327,3,0,13,202279,3,8.0,5,9,prior
1,3,24838,2,1,16,205970,16,12.0,5,17,prior
2,3,21903,4,1,4,205970,16,12.0,5,17,prior
3,4,46842,1,0,3,178520,36,7.0,1,9,prior
4,4,39758,3,1,19,178520,36,7.0,1,9,prior


In [58]:
transaction_log.describe(include='all')

,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,order_number,days_since_prior_order,order_dow,order_hour_of_day,eval_set
count,3.243449e+06,3.243449e+06,3.243449e+06,3.243449e+06,3.243449e+06,3.243449e+06,3.243449e+06,3.035420e+06,3.243449e+06,3.243449e+06,3243449
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,prior
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3243449
mean,1.711060e+06,2.557559e+04,8.351998e+00,5.890868e-01,9.925284e+00,1.029093e+05,1.712567e+01,1.110739e+01,2.738101e+00,1.342053e+01,NaN
std,9.872380e+05,1.409445e+04,7.127327e+00,4.919996e-01,6.281691e+00,5.947733e+04,1.752069e+01,8.781646e+00,2.089354e+00,4.248505e+00,NaN
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,8.563660e+05,1.353500e+04,3.000000e+00,0.000000e+00,4.000000e+00,5.136300e+04,5.000000e+00,5.000000e+00,1.000000e+00,1.000000e+01,NaN
50%,1.710730e+06,2.524400e+04,6.000000e+00,1.000000e+00,9.000000e+00,1.025410e+05,1.100000e+01,8.000000e+00,3.000000e+00,1.300000e+01,NaN
75%,2.566144e+06,3.791900e+04,1.100000e+01,1.000000e+00,1.600000e+01,1.543720e+05,2.400000e+01,1.500000e+01,5.000000e+00,1.600000e+01,NaN


In [59]:
transaction_log.isnull().sum()

order_id                       0
product_id                     0
add_to_cart_order              0
reordered                      0
department_id                  0
user_id                        0
order_number                   0
days_since_prior_order    208029
order_dow                      0
order_hour_of_day              0
eval_set                       0
dtype: int64

In [65]:
# View dataframe of only null days_simce_prior_order
transaction_log[transaction_log['days_since_prior_order'].isnull()]

,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,order_number,days_since_prior_order,order_dow,order_hour_of_day
16,20,35430,1,0,19,182912,1,NaN,6,17
17,20,7419,3,0,4,182912,1,NaN,6,17
37,35,28413,1,0,4,202527,1,NaN,1,13
48,57,47231,2,0,7,34016,1,NaN,6,8
49,57,43122,4,0,4,34016,1,NaN,6,8
...,...,...,...,...,...,...,...,...,...,...
3243278,3420914,43129,4,0,13,95068,1,NaN,4,13
3243280,3420918,19660,15,0,7,205643,1,NaN,6,23
3243420,3421044,14678,3,0,1,35140,1,NaN,4,15
3243446,3421081,38185,1,0,19,117076,1,NaN,3,11


In [68]:
# convert days_since_prior_order to int
transaction_log['days_since_prior_order'] = transaction_log['days_since_prior_order'].fillna(0).astype(int)

this exploration shows that NaN values in days_since_prior_orders indeed reflect every customer's first order

In [60]:
# filtering out test/train sets
print(transaction_log[transaction_log['eval_set'] == 'train'])
transaction_log[transaction_log['eval_set'] == 'test']

Empty DataFrame
Columns: [order_id, product_id, add_to_cart_order, reordered, department_id, user_id, order_number, days_since_prior_order, order_dow, order_hour_of_day, eval_set]
Index: []


,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,order_number,days_since_prior_order,order_dow,order_hour_of_day,eval_set


somehow, every test/train eval_set was not selected during the sampling. hm

In [61]:
# drop eval_set column as it's not needed for analysis
transaction_log.drop(columns=['eval_set'], inplace=True)

In [62]:
# checking for duplicates
transaction_log.duplicated().sum()

0

In [66]:
# Renaming key columns to match Fintech Framing
transaction_log.rename(columns={
    'user_id': 'Customer_ID',
    'order_id': 'Transaction_ID',
    'department_id': 'Merchant_Vertical_ID',
    'product_id': 'Product_SKU' 
}, inplace=True)


In [72]:
# Select and order final columns for clarity
final_transaction_log = transaction_log[['Customer_ID', 'Transaction_ID', 'order_number', 'days_since_prior_order', 'order_dow', 'order_hour_of_day', 'Merchant_Vertical_ID', 'Product_SKU']]

In [75]:
final_transaction_log.head()

,Customer_ID,Transaction_ID,order_number,days_since_prior_order,order_dow,order_hour_of_day,Merchant_Vertical_ID,Product_SKU
0,202279,2,3,8,5,9,13,9327
1,205970,3,16,12,5,17,16,24838
2,205970,3,16,12,5,17,4,21903
3,178520,4,36,7,1,9,3,46842
4,178520,4,36,7,1,9,19,39758


In [76]:
# Save the final, clean, merged log to the data folder
OUTPUT_FILE = os.path.join(DATA_PATH, 'processed_features.csv')
final_transaction_log.to_csv(OUTPUT_FILE, index=False)

In [78]:
print(f"Final shape of transaction log: {final_transaction_log.shape}")


Final shape of transaction log: (3243449, 8)
